In [ ]:
from collections import defaultdict

import matplotlib.pyplot as plt
import networkx as nx
import pyomo.environ as pe
import pyomo.opt as po

## Define the Graph

In [ ]:
nodes = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
edges = {(0, 1), (0, 2), (0, 3), (1, 4),
         (1, 6), (2, 1), (2, 3), (2, 5),
         (3, 5), (4, 2), (5, 7), (5, 8),
         (6, 4), (6, 7), (6, 9), (7, 4),
         (7, 9), (8, 3), (8, 7), (8, 9)}
distances = {(0, 1): 40, (0, 2):  8, (0, 3): 10, (1, 4):  6,
             (1, 6): 10, (2, 1):  4, (2, 3): 12, (2, 5):  2,
             (3, 5):  1, (4, 2):  2, (5, 7):  4, (5, 8):  3,
             (6, 4):  8, (6, 7): 20, (6, 9):  1, (7, 4):  0,
             (7, 9): 20, (8, 3):  6, (8, 7): 10, (8, 9):  2}

## Visualize Graph with NetworkX

In [ ]:
graph = nx.DiGraph()
graph.add_nodes_from(list(nodes))
graph.add_edges_from(list(edges))
pos = ({0: (4, 0), 1: (0, 0), 2: (4, 2), 3: (8, 0), 4: (2, 4),
        5: (6, 4), 6: (0, 8), 7: (4, 6), 8: (8, 8), 9: (4, 8)})

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))
nx.draw_networkx_nodes(graph, pos=pos, ax=ax, node_color='lightgray',
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(graph, pos=pos, ax=ax, labels=dict(zip(nodes, nodes)),
                        font_size=20)
nx.draw_networkx_edges(graph, pos=pos, ax=ax, node_size=2000, arrowsize=25)
nx.draw_networkx_edge_labels(graph, pos=pos, ax=ax, edge_labels=distances,
                             font_size=16, rotate=False)
plt.show()

## Define Sets of Incoming and Outgoing Edges
This data preprocessing step makes our lives easier when we define the flow balance constraints. For large graphs, doing this ahead of time in preprocessing (as opposed to on-the-fly during model construction) reduces model build time from $\mathcal{O}(n^2)$ and $\mathcal{O}(n)$.

In [ ]:
Vm = defaultdict(set)
Vp = defaultdict(set)
for (i, j) in edges:
    Vm[i].add(j)
    Vp[j].add(i)

## Formulate the Shortest Path MIP

Let $\mathcal{G} = (\mathcal{V}, \mathcal{E})$ be the graph. Let $s$ and $t$ be the start and terminal nodes, respectively, and let $d_{i,j}$ denote distance from node $i$ to node $j$. As variables, let $x_{i,j}$ be a 0-1 variable indicating the decision to permit flow across edge $(i, j)$. We also introduce the following notation to describe adjacent nodes on incoming and outgoing edges.

\begin{aligned}
    \mathcal{V}^-(i) = \{j\ |\ (i, j) \in \mathcal{E}\} \\
    \mathcal{V}^+(i) = \{j\ |\ (j, i) \in \mathcal{E}\}
\end{aligned}

We require one unit of flow to enter the graph at start node $s$ and one unit of flow to leave the graph at terminal node $t$. At every other node, we require flow balance. Our objective is to minimize the length of the path satisfying the constraints. That is, we find the distance-weighted sum of the $x_{i,j}$ variables, the shortest path.

$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{(i, j) \in \mathcal{E}} d_{ij} x_{ij} && \\
\text{subject to  }
& \sum_{j \in \mathcal{V}^-(s)} x_{sj} = 1 && \\
& \sum_{j \in \mathcal{V}^+(t)} x_{jt} = 1 && \\
& \sum_{j \in \mathcal{V}^+(i)} x_{ji} = \sum_{j \in \mathcal{V}^-(i)} x_{ij},
&& \qquad \forall i \in \mathcal{V} \setminus \{s, t\} \\
& x_{ij} \in \{0,1\}, 
&& \qquad \forall (i, j) \in \mathcal{E}
\end{alignat*}
$$

## Create the `instance` Object

In [ ]:
instance = pe.ConcreteModel()

## Define the Sets
Notice use of `within` when defining the set of edges. This ensures that $\mathcal{E} \subseteq \mathcal{V} \times \mathcal{V}$. While this is not necessary, it is good practice and will help identify data errors.

In [ ]:
instance.nodes = pe.Set(initialize=nodes)
instance.edges = pe.Set(within=instance.nodes*instance.nodes, initialize=edges)

## Define the Parameters
The first argument of `pe.Param` is the set in which the parameters are indexed. For example, distances are defined on the set of edges. The `initialize` argument here should be a dictionary-like construct. The dictionary keys should match align with the index set and the values are... well... the values. Notice also that $\mathcal{V}^+(\cdot)$ and $\mathcal{V}^-(\cdot)$ are indexed *sets*, not technically *parameters*. Pyomo does not directly support "jagged" sets like this, but using `pe.Param` is one way to implement such a feature.

In [ ]:
instance.Vm = pe.Param(instance.nodes, initialize=Vm, default=set(), within=pe.Any)
instance.Vp = pe.Param(instance.nodes, initialize=Vp, default=set(), within=pe.Any)
instance.s = 0
instance.t = 9
instance.distances = pe.Param(instance.edges, initialize=distances)

# Define the Variables
Just as it was with `pe.Param`, the first argument to `pe.Var` is the index set in which the variables are indexed. Optionally, we may define the domain of the variable set. Here, we require each $x_{i,j}$ to be binary -- either 0 or 1. If not explicitly defined, the domain is assumed to be the set of all real numbers (`pe.Reals`).

In [ ]:
instance.x = pe.Var(instance.edges, domain=pe.Binary)

As you will learn in IP, the constraint matrix is totally unimodular. Consequently, the variables will assume integer values even if they are relaxed to the $[0, 1]$ interval. Below, we implement the relaxation using a combination of `domain` and `bounds`.

In [ ]:
# totally unimodular constraint matrix
# instance.x = pe.Var(instance.edges, domain=pe.Reals, bounds=(0, 1))

## Define the Objective

In [ ]:
def shortest_path(instance):
    return sum(instance.distances[i, j] * instance.x[i, j]
               for (i, j) in instance.edges)

instance.shortest_path = pe.Objective(sense=pe.minimize, rule=shortest_path)

In [ ]:
# instance.shortest_path = pe.Objective()
# instance.shortest_path.sense = pe.minimize
# obj_expr = sum(instance.distances[i, j] * instance.x[i, j]
#                for (i, j) in instance.edges)
# instance.shortest_path.expr = obj_expr

## Define the Constraints

In [ ]:
def flow_balance(instance, i):
    flow_in = sum([instance.x[j, i] for j in instance.Vp[i]])
    flow_out = sum([instance.x[i, j] for j in instance.Vm[i]])
    if i == instance.s:
        constraint = (flow_out == 1)
    elif i == instance.t:
        constraint = (flow_in == 1)
    else:
        constraint = (flow_in == flow_out)
    return constraint

instance.flow_balance = pe.Constraint(instance.nodes, rule=flow_balance)

In [ ]:
# instance.flow_balance = pe.ConstraintList()
# for i in instance.nodes:
#     if i == instance.s:
#         lhs = sum(instance.x[i, j] for j in instance.Vm[i])
#         rhs = 1
#     elif i == instance.t:
#         lhs = sum(instance.x[j, i] for j in instance.Vp[i])
#         rhs = 1
#     else:
#         lhs = sum(instance.x[i, j] for j in instance.Vm[i])
#         rhs = sum(instance.x[j, i] for j in instance.Vp[i])
#     instance.flow_balance.add(lhs == rhs)

In [ ]:
solver = po.SolverFactory('glpk') # 'glpk', 'gurobi', 'bonmin', 'mosek'
result = solver.solve(instance)

Interpret the output to determine the shortest path.

In [ ]:
i = int(instance.s)
path_nodes = [i]
path_edges = []
stop = False
while not stop:
    for j in delta_neg[i]:
        if instance.x[i, j].value == 1:
            if j == int(instance.t):
                stop = True
            path_nodes.append(j)
            path_edges.append((i, j))
            i = j
            break

display(path_nodes)

## Visualize Solution with NetworkX

In [ ]:
node_colors = ['lightblue' if i in path_nodes else 'lightgray'
               for i in graph.nodes]
edge_colors = ['blue' if (i, j) in path_edges else 'black'
               for (i, j) in graph.edges]
edge_widths = [2 if (i, j) in path_edges else 1
               for (i, j) in graph.edges]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

nx.draw_networkx_nodes(graph, pos=pos, ax=ax, node_color=node_colors,
                       edgecolors='black', node_size=2000)
nx.draw_networkx_labels(graph, pos=pos, ax=ax, labels=dict(zip(nodes, nodes)),
                        font_size=20)
nx.draw_networkx_edges(graph, pos=pos, ax=ax, node_size=2000, arrowsize=25,
                       edge_color=edge_colors, width=edge_widths)
nx.draw_networkx_edge_labels(graph, pos=pos, ax=ax, edge_labels=distances,
                             font_size=16, rotate=False)
nx.draw_networkx_edges
plt.show()